In [1]:
# Importando bibliotecas que serao utilizadas neste projeto
import pandas as pd
import numpy as np

from itertools import product
from multiprocessing import Pool
from scipy.stats import kurtosis, skew
from scipy.optimize import minimize
import scipy.stats as scs
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm

# Stats
from scipy import stats
from scipy.stats import skew, norm
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import datetime
from dateutil.relativedelta import relativedelta 
import time
import gc
import os
from tqdm import tqdm_notebook

# Ignorar warnings
import warnings
warnings.filterwarnings(action="ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = False

# Variavel para indicar o path local
LOCAL_DATA_FOLDER  = './data/'

In [2]:
from sklearn import preprocessing, metrics
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LinearRegression

# lib de modelos de machine learning
import xgboost as xgb
import lightgbm as lgb

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
#KAGGLE_DATA_FOLDER = '/kaggle/input/m5-forecasting-accuracy/'

In [6]:
calendar = pd.read_csv('./data/calendar.csv')
sell_prices = pd.read_csv('./data/sell_prices.csv')
sales_train_validation = pd.read_csv('./data/sales_train_validation.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [7]:
calendar = reduce_mem_usage(calendar)
sell_prices = reduce_mem_usage(sell_prices)
sales_train_validation = reduce_mem_usage(sales_train_validation)
submission = reduce_mem_usage(submission)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)


In [8]:
sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                                     var_name = 'day',
                                     value_name = 'demand')
print('Melted sales train validation tem {} linhas e {} colunas'.format(sales_train_validation.shape[0],
                                                                            sales_train_validation.shape[1]))

Melted sales train validation tem 58327370 linhas e 8 colunas


In [9]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [10]:
test_rows = [row for row in submission['id'] if 'validation' in row]
val_rows = [row for row in submission['id'] if 'evaluation' in row]

In [11]:
test_rows[:3]

['HOBBIES_1_001_CA_1_validation',
 'HOBBIES_1_002_CA_1_validation',
 'HOBBIES_1_003_CA_1_validation']

In [12]:
val_rows[:3]

['HOBBIES_1_001_CA_1_evaluation',
 'HOBBIES_1_002_CA_1_evaluation',
 'HOBBIES_1_003_CA_1_evaluation']

In [13]:
test = submission[submission['id'].isin(test_rows)]
val = submission[submission['id'].isin(val_rows)]

In [14]:
test

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30486,FOODS_3_824_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30487,FOODS_3_825_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30488,FOODS_3_826_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
val

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30491,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30492,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30493,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30494,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
test.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 
                'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 
                'd_1931', 'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 
                'd_1940', 'd_1941']
val.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 
               'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 
               'd_1959', 'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 
               'd_1968', 'd_1969']

In [17]:
len(test.columns)

29

In [18]:
len(val.columns)

29

In [19]:
product = sales_train_validation[['id',
                                  'item_id',
                                  'dept_id',
                                  'cat_id',
                                  'store_id',
                                  'state_id']].drop_duplicates()

In [20]:
product.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [21]:
test = test.merge(product, how = 'left', on = 'id')
val = val.merge(product, how = 'left', on = 'id')

In [22]:
test.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [23]:
val.head()

,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [24]:
test = pd.melt(test, id_vars = ['id', 
                                'item_id',
                                'dept_id',
                                'cat_id',
                                'store_id',
                                'state_id'], 
               var_name = 'day', 
               value_name = 'demand')

val = pd.melt(val, id_vars = ['id', 
                              'item_id',
                              'dept_id',
                              'cat_id',
                              'store_id', 
                              'state_id'], 
              var_name = 'day', 
              value_name = 'demand')

In [25]:
test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0


In [26]:
val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0
1,HOBBIES_1_002_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0
2,HOBBIES_1_003_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0
3,HOBBIES_1_004_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0
4,HOBBIES_1_005_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0


In [27]:
sales_train_validation['part'] = 'train'
test['part'] = 'test'
val['part'] = 'val'

In [28]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train


In [29]:
test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,test
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,test
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,test
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,test
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,0,test


In [30]:
val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0,val
1,HOBBIES_1_002_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0,val
2,HOBBIES_1_003_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0,val
3,HOBBIES_1_004_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0,val
4,HOBBIES_1_005_CA_1_evaluation,NaN,NaN,NaN,NaN,NaN,d_1942,0,val


In [31]:
data = pd.concat([sales_train_validation, test, val], axis = 0)

In [32]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train


In [33]:
del sales_train_validation, test, val

In [34]:
data = data.loc[45000000:]

In [35]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
45000000,FOODS_3_634_WI_2_validation,FOODS_3_634,FOODS_3,FOODS,WI_2,WI,d_1476,7,train
45000001,FOODS_3_635_WI_2_validation,FOODS_3_635,FOODS_3,FOODS,WI_2,WI,d_1476,8,train
45000002,FOODS_3_636_WI_2_validation,FOODS_3_636,FOODS_3,FOODS,WI_2,WI,d_1476,12,train
45000003,FOODS_3_637_WI_2_validation,FOODS_3_637,FOODS_3,FOODS,WI_2,WI,d_1476,1,train
45000004,FOODS_3_638_WI_2_validation,FOODS_3_638,FOODS_3,FOODS,WI_2,WI,d_1476,0,train


In [36]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
853715,FOODS_3_823_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,d_1969,0,val
853716,FOODS_3_824_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,d_1969,0,val
853717,FOODS_3_825_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,d_1969,0,val
853718,FOODS_3_826_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,d_1969,0,val
853719,FOODS_3_827_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,d_1969,0,val


In [37]:
data = data[data['part'] != 'val']

In [38]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [39]:
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])

In [40]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,FOODS_3_634_WI_2_validation,FOODS_3_634,FOODS_3,FOODS,WI_2,WI,d_1476,7,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,NaN,NaN,NaN,NaN,0,1,1
1,FOODS_3_635_WI_2_validation,FOODS_3_635,FOODS_3,FOODS,WI_2,WI,d_1476,8,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,NaN,NaN,NaN,NaN,0,1,1
2,FOODS_3_636_WI_2_validation,FOODS_3_636,FOODS_3,FOODS,WI_2,WI,d_1476,12,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,NaN,NaN,NaN,NaN,0,1,1
3,FOODS_3_637_WI_2_validation,FOODS_3_637,FOODS_3,FOODS,WI_2,WI,d_1476,1,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,NaN,NaN,NaN,NaN,0,1,1
4,FOODS_3_638_WI_2_validation,FOODS_3_638,FOODS_3,FOODS,WI_2,WI,d_1476,0,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,NaN,NaN,NaN,NaN,0,1,1


In [41]:
nan_features_cat = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features_cat:
    data[feature].fillna('unknown', inplace = True)

In [42]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,FOODS_3_634_WI_2_validation,FOODS_3_634,FOODS_3,FOODS,WI_2,WI,d_1476,7,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,unknown,unknown,unknown,unknown,0,1,1
1,FOODS_3_635_WI_2_validation,FOODS_3_635,FOODS_3,FOODS,WI_2,WI,d_1476,8,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,unknown,unknown,unknown,unknown,0,1,1
2,FOODS_3_636_WI_2_validation,FOODS_3_636,FOODS_3,FOODS,WI_2,WI,d_1476,12,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,unknown,unknown,unknown,unknown,0,1,1
3,FOODS_3_637_WI_2_validation,FOODS_3_637,FOODS_3,FOODS,WI_2,WI,d_1476,1,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,unknown,unknown,unknown,unknown,0,1,1
4,FOODS_3_638_WI_2_validation,FOODS_3_638,FOODS_3,FOODS,WI_2,WI,d_1476,0,train,2015-02-12,11502,Thursday,6,2,2015,d_1476,unknown,unknown,unknown,unknown,0,1,1


In [43]:
data.drop(['d', 'day', 'weekday'], inplace = True, axis = 1)
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
print('Dataset final para treino tem {} linhas e {} colunas'.format(data.shape[0], data.shape[1]))

Dataset final para treino tem 14181090 linhas e 21 colunas


In [44]:
data.isnull().sum()

id                  0
item_id             0
dept_id             0
cat_id              0
store_id            0
state_id            0
demand              0
part                0
date                0
wm_yr_wk            0
wday                0
month               0
year                0
event_name_1        0
event_type_1        0
event_name_2        0
event_type_2        0
snap_CA             0
snap_TX             0
snap_WI             0
sell_price      68904
dtype: int64

In [45]:
data['sell_price'].fillna(0, inplace = True)

In [46]:
encoder = preprocessing.LabelEncoder()

In [47]:
data['id']

0           FOODS_3_634_WI_2_validation
1           FOODS_3_635_WI_2_validation
2           FOODS_3_636_WI_2_validation
3           FOODS_3_637_WI_2_validation
4           FOODS_3_638_WI_2_validation
                       ...             
14181085    FOODS_3_823_WI_3_validation
14181086    FOODS_3_824_WI_3_validation
14181087    FOODS_3_825_WI_3_validation
14181088    FOODS_3_826_WI_3_validation
14181089    FOODS_3_827_WI_3_validation
Name: id, Length: 14181090, dtype: object

In [48]:
data['id_encode'] = encoder.fit_transform(data['id'])

In [49]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,id_encode
0,FOODS_3_634_WI_2_validation,FOODS_3_634,FOODS_3,FOODS,WI_2,WI,7,train,2015-02-12,11502,6,2,2015,unknown,unknown,unknown,unknown,0,1,1,3.980469,12468
1,FOODS_3_635_WI_2_validation,FOODS_3_635,FOODS_3,FOODS,WI_2,WI,8,train,2015-02-12,11502,6,2,2015,unknown,unknown,unknown,unknown,0,1,1,0.939941,12478
2,FOODS_3_636_WI_2_validation,FOODS_3_636,FOODS_3,FOODS,WI_2,WI,12,train,2015-02-12,11502,6,2,2015,unknown,unknown,unknown,unknown,0,1,1,3.500000,12488
3,FOODS_3_637_WI_2_validation,FOODS_3_637,FOODS_3,FOODS,WI_2,WI,1,train,2015-02-12,11502,6,2,2015,unknown,unknown,unknown,unknown,0,1,1,2.980469,12498
4,FOODS_3_638_WI_2_validation,FOODS_3_638,FOODS_3,FOODS,WI_2,WI,0,train,2015-02-12,11502,6,2,2015,unknown,unknown,unknown,unknown,0,1,1,1.450195,12508


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14181090 entries, 0 to 14181089
Data columns (total 22 columns):
id              object
item_id         object
dept_id         object
cat_id          object
store_id        object
state_id        object
demand          int16
part            object
date            object
wm_yr_wk        int16
wday            int8
month           int8
year            int16
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA         int8
snap_TX         int8
snap_WI         int8
sell_price      float16
id_encode       int32
dtypes: float16(1), int16(3), int32(1), int8(5), object(12)
memory usage: 1.6+ GB


In [51]:
cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
           'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

In [52]:
for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])

In [53]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,id_encode
0,FOODS_3_634_WI_2_validation,1246,2,0,8,2,7,train,2015-02-12,11502,6,2,2015,30,4,0,0,0,1,1,3.980469,12468
1,FOODS_3_635_WI_2_validation,1247,2,0,8,2,8,train,2015-02-12,11502,6,2,2015,30,4,0,0,0,1,1,0.939941,12478
2,FOODS_3_636_WI_2_validation,1248,2,0,8,2,12,train,2015-02-12,11502,6,2,2015,30,4,0,0,0,1,1,3.500000,12488
3,FOODS_3_637_WI_2_validation,1249,2,0,8,2,1,train,2015-02-12,11502,6,2,2015,30,4,0,0,0,1,1,2.980469,12498
4,FOODS_3_638_WI_2_validation,1250,2,0,8,2,0,train,2015-02-12,11502,6,2,2015,30,4,0,0,0,1,1,1.450195,12508


In [54]:
gc.collect()

260

In [55]:
print(min(data['date']), max(data['date']))

2015-02-12 2016-05-22


In [56]:
data.to_csv("./data/data_eng.csv")